In [1]:
!pip install 'vanna[openai]'

  Obtaining dependency information for vanna[openai] from https://files.pythonhosted.org/packages/0f/e4/af7dfaf8e103acb90f5d169144d2e9994e17d9c7992a5881f1a81148dcac/vanna-0.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for kaleido from https://files.pythonhosted.org/packages/e0/f7/0ccaa596ec341963adbb4f839774c36d5659e75a0812d946732b927d480e/kaleido-0.2.1-py2.py3-none-macosx_10_11_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 MB 158.8 kB/s eta 0:00:00m eta 0:00:010:00:15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 876.2 kB/s eta 0:00:001m1.3 MB/s eta 0:00:01


In [1]:
from vanna.openai.openai_chat import OpenAI_Chat
from vanna.vannadb.vannadb_vector import VannaDB_VectorStore
import pandas as pd
import os

In [2]:
openai_key = os.getenv('OPENAI_API_KEY')
my_vanna_api = '1a642e16571c4c1fb05343ccbe509078'
my_vanna_model = 'comp0078-test'

In [3]:
class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = 'comp0078-test'
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=my_vanna_api, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': openai_key, 'model': 'gpt-3.5-turbo'})

In [4]:
vn.connect_to_sqlite('/Users/marceloyou/Desktop/UCL-DSML/COMP0087-Boss/SQLess/src/evaluation/database/academic/academic.sqlite')

In [5]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

In [6]:
df_ddl

,type,sql
0,table,"CREATE TABLE ""author"" (\n""aid"" int,\n""homepage..."
1,table,"CREATE TABLE ""conference"" (\n""cid"" int,\n""home..."
2,table,"CREATE TABLE ""domain"" (\n""did"" int,\n""name"" te..."
3,table,"CREATE TABLE ""domain_author"" (\n""aid"" int,\n""d..."
4,table,"CREATE TABLE ""domain_conference"" (\n""cid"" int,..."
5,table,"CREATE TABLE ""journal"" (\n""homepage"" text,\n""j..."
6,table,"CREATE TABLE ""domain_journal"" (\n""did"" int,\n""..."
7,table,"CREATE TABLE ""keyword"" (\n""keyword"" text,\n""ki..."
8,table,"CREATE TABLE ""domain_keyword"" (\n""did"" int,\n""..."
9,table,"CREATE TABLE ""publication"" (\n""abstract"" text,..."


In [11]:
for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE "author" (
"aid" int,
"homepage" text,
"name" text,
"oid" int,
primary key("aid"),
foreign key("oid") references "organization"("oid"))
Adding ddl: CREATE TABLE "conference" (
"cid" int,
"homepage" text,
"name" text,
primary key ("cid")
)
Adding ddl: CREATE TABLE "domain" (
"did" int,
"name" text,
primary key ("did")
)
Adding ddl: CREATE TABLE "domain_author" (
"aid" int,
"did" int,
primary key ("did", "aid"),
foreign key("did") references "domain"("did"),
foreign key("aid") references "author"("aid"))
Adding ddl: CREATE TABLE "domain_conference" (
"cid" int,
"did" int,
primary key ("did", "cid"),
foreign key("did") references "domain"("did"),
foreign key("cid") references "conference"("cid"))
Adding ddl: CREATE TABLE "journal" (
"homepage" text,
"jid" int,
"name" text,
primary key("jid")
)
Adding ddl: CREATE TABLE "domain_journal" (
"did" int,
"jid" int,
primary key ("did", "jid"),
foreign key("did") references "domain"("did"),
foreign key("jid") references "j

In [7]:
vn.get_training_data()

,id,training_data_type,question,content
0,48221-ddl,ddl,NaN,"CREATE TABLE ""domain_author"" (\n""aid"" int,\n""d..."
1,48222-ddl,ddl,NaN,"CREATE TABLE ""domain_conference"" (\n""cid"" int,..."
2,48229-ddl,ddl,NaN,"CREATE TABLE ""organization"" (\n""continent"" tex..."
3,48224-ddl,ddl,NaN,"CREATE TABLE ""domain_journal"" (\n""did"" int,\n""..."
4,48231-ddl,ddl,NaN,"CREATE TABLE ""writes"" (\n""aid"" int,\n""pid"" int..."
5,48223-ddl,ddl,NaN,"CREATE TABLE ""journal"" (\n""homepage"" text,\n""j..."
6,48230-ddl,ddl,NaN,"CREATE TABLE ""publication_keyword"" (\n""pid"" in..."
7,48225-ddl,ddl,NaN,"CREATE TABLE ""keyword"" (\n""keyword"" text,\n""ki..."
8,48227-ddl,ddl,NaN,"CREATE TABLE ""publication"" (\n""abstract"" text,..."
9,48226-ddl,ddl,NaN,"CREATE TABLE ""domain_keyword"" (\n""did"" int,\n""..."


In [8]:
questions = pd.read_csv('/Users/marceloyou/Desktop/UCL-DSML/COMP0087-Boss/SQLess/data/sql_test_suite_academic/masked_academic_sample.csv')

In [9]:
questions

,question,db_id,db_path,masked
0,return me the year of Making database systems ...,academic,database/academic/academic.sqlite,return me the [MASK] of Making database system...
1,return me the papers after 2000 .,academic,database/academic/academic.sqlite,return me the papers after 2000 .
2,return me all the keywords .,academic,database/academic/academic.sqlite,return me all the [MASK] .
3,return me the references of Making database sy...,academic,database/academic/academic.sqlite,return me the [MASK] of Making database system...
4,return me the authors who have papers in VLDB ...,academic,database/academic/academic.sqlite,return me the [MASK] who have papers in VLDB [...
5,return me the area of PVLDB .,academic,database/academic/academic.sqlite,return me the area of PVLDB .
6,"return me the journals, which have papers by H...",academic,database/academic/academic.sqlite,return me the [MASK] which have papers by H. V...
7,return me the papers on PVLDB after 2000 .,academic,database/academic/academic.sqlite,return me the papers on PVLDB after 2000 .
8,return me the papers by H. V. Jagadish after 2...,academic,database/academic/academic.sqlite,return me the papers by H. V. Jagadish after 2...
9,return me the keywords of Making database syst...,academic,database/academic/academic.sqlite,return me the [MASK] of Making database system...


In [10]:
res = []
import tqdm

for q in tqdm.tqdm(questions.question.to_list()):
    ans = vn.ask(question=q, print_results=False, auto_train=False, visualize=False)
    res.append(ans[0])

  0%|          | 0/40 [00:00<?, ?it/s]

[{'role': 'system', 'content': 'The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use responses to past questions also to guide you:\n\nCREATE TABLE "publication" (\n"abstract" text,\n"cid" text,\n"citation_num" int,\n"jid" int,\n"pid" int,\n"reference_num" int,\n"title" text,\n"year" int,\nprimary key("pid"),\nforeign key("cid") references "conference"("cid"),\nforeign key("jid") references "journal"("jid"))\n\nCREATE TABLE "author" (\n"aid" int,\n"homepage" text,\n"name" text,\n"oid" int,\nprimary key("aid"),\nforeign key("oid") references "organization"("oid"))\n\nCREATE TABLE "cite" (\n"cited" int,\n"citing" int,\nforeign key("citing") references "publication"("pid"),\nforeign key("cited") references "publication"("pid"))\n\nCREATE TABLE "

  2%|▎         | 1/40 [00:02<01:33,  2.39s/it]

```sql
SELECT year
FROM publication
WHERE title = 'Making database systems usable'
```
Output from LLM: ```sql
SELECT year
FROM publication
WHERE title = 'Making database systems usable'
``` 
Extracted SQL: SELECT year
FROM publication
WHERE title = 'Making database systems usable'

[{'role': 'system', 'content': 'The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use responses to past questions also to guide you:\n\nCREATE TABLE "publication" (\n"abstract" text,\n"cid" text,\n"citation_num" int,\n"jid" int,\n"pid" int,\n"reference_num" int,\n"title" text,\n"year" int,\nprimary key("pid"),\nforeign key("cid") references "conference"("cid"),\nforeign key("jid") references "journal"("jid"))\n\nCREATE TABLE "cite" (\n"cited" int,\n"citing" int,\nf

  5%|▌         | 2/40 [00:03<01:11,  1.88s/it]

...

```sql
WITH paper_citations AS (
    SELECT p.pid, p.title, p.citation_num
    FROM publication p
    JOIN domain_publication dp ON p.pid = dp.pid
    JOIN domain d ON dp.did = d.did
    WHERE d.name = 'Databases'
)
SELECT pid, title, citation_num
FROM paper_citations
ORDER BY citation_num DESC
LIMIT 1;
```
Output from LLM: ```sql
WITH paper_citations AS (
    SELECT p.pid, p.title, p.citation_num
    FROM publication p
    JOIN domain_publication dp ON p.pid = dp.pid
    JOIN domain d ON dp.did = d.did
    WHERE d.name = 'Databases'
)
SELECT pid, title, citation_num
FROM paper_citations
ORDER BY citation_num DESC
LIMIT 1;
``` 
Extracted SQL: WITH paper_citations AS (
    SELECT p.pid, p.title, p.citation_num
    FROM publication p
    JOIN domain_publication dp ON p.pid = dp.pid
    JOIN domain d ON dp.did = d.did
    WHERE d.name = 'Databases'
)
SELECT pid, title, citation_num
FROM paper_citations
ORDER BY citation_num DESC
LIMIT 1;

[{'role': 'system', 'content': 'The user provides 

 98%|█████████▊| 39/40 [01:37<00:02,  2.62s/it]

```sql
SELECT p.title
FROM publication p
JOIN author a ON p.pid = a.pid
WHERE a.name = 'H. V. Jagadish'
ORDER BY p.citation_num DESC
LIMIT 1;
```
Output from LLM: ```sql
SELECT p.title
FROM publication p
JOIN author a ON p.pid = a.pid
WHERE a.name = 'H. V. Jagadish'
ORDER BY p.citation_num DESC
LIMIT 1;
``` 
Extracted SQL: SELECT p.title
FROM publication p
JOIN author a ON p.pid = a.pid
WHERE a.name = 'H. V. Jagadish'
ORDER BY p.citation_num DESC
LIMIT 1;

Couldn't run sql:  Execution failed on sql 'SELECT p.title
FROM publication p
JOIN author a ON p.pid = a.pid
WHERE a.name = 'H. V. Jagadish'
ORDER BY p.citation_num DESC
LIMIT 1;
': no such column: a.pid
[{'role': 'system', 'content': 'The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use re

100%|██████████| 40/40 [01:39<00:00,  2.48s/it]

```sql
SELECT p.*
FROM publication p
WHERE p.year > 2000
AND p.cid = (SELECT cid FROM conference WHERE name = 'PVLDB')
ORDER BY p.citation_num DESC
LIMIT 1;
```
Output from LLM: ```sql
SELECT p.*
FROM publication p
WHERE p.year > 2000
AND p.cid = (SELECT cid FROM conference WHERE name = 'PVLDB')
ORDER BY p.citation_num DESC
LIMIT 1;
``` 
Extracted SQL: SELECT p.*
FROM publication p
WHERE p.year > 2000
AND p.cid = (SELECT cid FROM conference WHERE name = 'PVLDB')
ORDER BY p.citation_num DESC
LIMIT 1;



In [11]:
len(res)

40

In [12]:
res2 = list(map(lambda x: x.replace('\n', ' '), res))

In [13]:
with open('predict.txt', 'w') as f:
    for r in res2:
        f.write(f'{r}\n')